# Octree introduction
---
Requirements:
* octree
* vtk >= 6.2.0

In [1]:
# Imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import vtk
import octree

In [2]:
print vtk.vtkVersion.GetVTKVersion()

6.2.0


## Load 3D model geometry (stl file)

In [100]:
# Read in stl file using vtk
reader = vtk.vtkSTLReader()
reader.SetFileName("knot.stl")
reader.MergingOn()
reader.Update()
stl = reader.GetOutput()
print "Number of points    = %d" % stl.GetNumberOfPoints()
print "Number of triangles = %d" % stl.GetNumberOfCells()

Number of points    = 38214
Number of triangles = 76428


In [4]:
# Extract polygon info from stl

# 1. Get array of point coordinates
numPoints   = stl.GetNumberOfPoints()
pointCoords = np.zeros((numPoints,3),dtype=float)
for i in xrange(numPoints):
    pointCoords[i,:] = stl.GetPoint(i)
    
# 2. Get polygon connectivity
numPolys     = stl.GetNumberOfCells()
connectivity = np.zeros((numPolys,3),dtype=np.int32)
for i in xrange(numPolys):
    atri = stl.GetCell(i)
    ids = atri.GetPointIds()
    for j in range(3):
        connectivity[i,j] = ids.GetId(j)

## Generate octree

In [5]:
# Create octree structure containing stl poly mesh
tree = octree.PyOctree(pointCoords,connectivity)

In [6]:
# Print out basic Octree data
print "Size of Octree               = %.3fmm" % tree.root.size
print "Number of Octnodes in Octree = %d" % tree.getNumberOfNodes()
print "Number of polys in Octree    = %d" % tree.numPolys

Size of Octree               = 3.115mm
Number of Octnodes in Octree = 2497
Number of polys in Octree    = 76428


## Explore tree nodes

In [103]:
# Get number of nodes in tree
tree.getNumberOfNodes()

2497

In [104]:
# Get the root node
tree.root

In [97]:
# Get the branches of the root node
tree.root.branches

All tree branches are given a unique id. For example:

* '0' is the root node
* '0-0' is the first branch of the root node
* '0-7' is the eight (and last) branch of the root node
* '0-0-0' is the first branch of octnode '0-0' etc.

In [99]:
# An octnode can be accessed using its id. For example:
tree.getNodeFromId('0-0')

In [47]:
# An octnode that has no branches is a leaf
print tree.getNodeFromId('0-0').isLeaf
print tree.getNodeFromId('0-0-0').isLeaf

False
True


## Explore polygons in tree

In [106]:
# Get list of first 10 polygons stored in tree
tree.polyList[0:10]

In [107]:
# Get details of the first polygon in the tree
tri = tree.polyList[0]
print 'Poly label = ', tri.label
print 'Vertex coordinates = (%s,%s,%s)'  % (tri.vertices[0], tri.vertices[1], tri.vertices[2] )
print 'Face normal direction = ', tri.N
print 'Perp. distance from origin = ', tri.D

Poly label =  0
Vertex coordinates = ([ 2.54317427  7.41368246  2.8599999 ],[ 2.51169538  7.40434027  2.8599999 ],[ 2.48648334  7.48027468  2.8599999 ])
Face normal direction =  [ 0.  0. -1.]
Perp. distance from origin =  -2.8599998951


## Find intersections between 3D object and ray

In [116]:
# Create a list containing a single ray
xs,xe,ys,ye,zs,ze = stl.GetBounds()
x = 0.5*np.mean([xs,xe])
y = np.mean([ys,ye])
rayPointList = np.array([[[x,y,zs],[x,y,ze]]],dtype=np.float32)

In [118]:
# Find if an intersection occurred
for i in tree.rayIntersections(rayPointList):
    print i==1

True


In [123]:
# Get intersection points for a single ray
ray = rayPointList[0]
for i in tree.rayIntersection(ray):
    print 'Coordinates = ', i.p, ',    Parametric distance along ray from origin = %.3f' % i.s

Coordinates =  [ 0.75257736  7.32612896  3.32434068] ,    Parametric distance along ray from origin = 0.464
Coordinates =  [ 0.75257736  7.32612896  3.64137301] ,    Parametric distance along ray from origin = 0.781
Coordinates =  [ 0.75257736  7.32612896  3.87418443] ,    Parametric distance along ray from origin = 1.014
Coordinates =  [ 0.75257736  7.32612896  4.41666534] ,    Parametric distance along ray from origin = 1.557


## Write out a vtk file of the octree structure 

In [126]:
# Create a vtk representation of Octree
print tree.getOctreeRep()

None
